In [123]:
import pandas as pd
import csv
import re

In [124]:
#Regular Info
Employee_Information_df = pd.read_csv(r'C:\Users\ASUS\Desktop\Payroll System\1 Information CSVs\Employee-Information.csv')
Standard_Clock_Hours_df = pd.read_csv(r'C:\Users\ASUS\Desktop\Payroll System\1 Information CSVs\Standard-Clock-Hours.csv')

#Input                                                                  #Name of the Input CSV
Day_df = pd.read_csv(r'C:\Users\ASUS\Desktop\Payroll System\2 Input CSVs\In-Monday.csv')

In [125]:
Employee_Information_df

,ID Number,First,Last,Date Started,Date Left,Rate Per Day,Rate Per Month
0,10000000,Angelo,Johnson,12/06/2018,0,537,16110
1,10000001,Bea,Ramirez,30/07/2015,0,600,18000
2,10000002,Chris,Evans,22/12/2018,0,840,25200
3,10000003,Dennis,Tan,04/12/2016,0,550,16500
4,10000004,Emily,Santos,14/08/2017,0,1045,31350
5,10000005,Frances,Mercado,15/03/2019,0,729,21870
6,10000006,Gary,Cruz,21/03/2016,0,1255,37650
7,10000007,Hans,Chan,16/05/2018,0,920,27600
8,10000008,Isabel,Juan,18/05/2017,0,620,18600
9,10000009,Jen,Ang,12/09/2018,0,845,25350


In [126]:
Standard_Clock_Hours_df

,Date,Clock In Time,Clock Out Time
0,Monday,8:00,16:00
1,Tuesday,8:00,16:00
2,Wednesday,8:00,16:00
3,Thursday,8:00,16:00
4,Friday,8:00,16:00
5,Saturday,8:00,16:00
6,Sunday,8:00,12:00


In [127]:
# Removes Colons of time in Standard Clock Hours and Input CSV
Day_df.loc[:,"Time In"] = Day_df.loc[:,"Time In"].apply(lambda x: re.sub(":","", x) )
Day_df.loc[:,"Time Out"] = Day_df.loc[:,"Time Out"].apply(lambda x: re.sub(":","", x) )

Standard_Clock_Hours_df.loc[:,"Clock In Time"] = Standard_Clock_Hours_df.loc[:,"Clock In Time"].apply(lambda x: re.sub(":","", x) )
Standard_Clock_Hours_df.loc[:,"Clock Out Time"] = Standard_Clock_Hours_df.loc[:,"Clock Out Time"].apply(lambda x: re.sub(":","", x) )

In [128]:
def time_to_minutes(x):
    if len(str(x))==3:
        hours = str(x)
        hours = int(hours[0])*60
        minutes = str(x)
        minutes = int(minutes[1:3])
        time = hours + minutes
        time = str(time)
        return time
    if len(str(x))==4:
        hours = str(x)
        hours = int(hours[0:2])*60
        minutes = str(x)
        minutes = int(minutes[2:4])
        time = hours + minutes
        time = str(time)
        return time

In [129]:
def time_round_up(x):
    hours = int(x)/60
    rational = float(hours)
    integer = int(hours)
    decimal = rational - integer
    if decimal >=0.5:
        roundnumber = integer + 1
        return str(roundnumber)
    else:
        return str(integer)

In [130]:
def work_hours(x):
    if int(x)<=8:
        work_hours = int(x)
        return work_hours
    else:
        work_hours = 8
        return work_hours
    return work_hours

In [131]:
def overtime_hours(x):
    if int(x)<=8:
        overtime_hours = 0
        return overtime_hours
    elif int(x)>8:
        overtime_hours = int(x) - 8
        return overtime_hours
    return overtime_hours

In [132]:
def late_hours(x):
    if int(x)<=0:
        late_hours = 0
        return late_hours
    elif int(x)>0:
        late_hours = int(x) / 60
        rational = float(late_hours)
        integer = int(late_hours)
        decimal = rational - integer
        if decimal >=0:
            late_hours = integer + 1
            return late_hours
        else:
            return integer

In [164]:
def debt_tracker(x):
    paid_debt = 0
    ID_Number = x
    directory = "C:\\Users\\ASUS\\Desktop\\Payroll System\\4 Debt Info\\"
    debt_file_name = directory + "DebtTracker_" + str(ID_Number) + ".csv"
    temp_df = pd.read_csv(debt_file_name)
    Debt_Tracker_df = temp_df.loc[:,"Debt Number":]
    
    for i in range(0,len(Debt_Tracker_df.loc[:,'Remaining Debt'])):
        if Debt_Tracker_df.iloc[i,1] > 0:
            Debt_Tracker_df.iloc[i,1] = Debt_Tracker_df.iloc[i,1] - Debt_Tracker_df.iloc[i,3]
            Debt_Tracker_df.iloc[i,2] = Debt_Tracker_df.iloc[i,2] - 1
            paid_debt += Debt_Tracker_df.iloc[i,3]
    
    Debt_Tracker_df.to_csv(debt_file_name)

    return paid_debt

In [165]:
#Reads a reference CSV to start a DataFrame
Out_DayX_df_Initial = pd.read_csv(r'C:\Users\ASUS\Desktop\Payroll System\5 Reference CSVs\Output-Reference.csv')

#Merge Salary Information
Employee_Wages_df = Employee_Information_df.loc[:,["ID Number","Rate Per Day"]]
Out_DayX_df = pd.merge(Employee_Wages_df,Out_DayX_df_Initial,on="ID Number",how="left")

#Input Information
Out_DayX_df.loc[:,"ID Number"] = Day_df.loc[:,"ID Number"]
Out_DayX_df.loc[:,"TI min"] = Day_df.loc[:,"Time In"].apply(lambda x: time_to_minutes(x))
Out_DayX_df.loc[:,"TO min"] = Day_df.loc[:,"Time Out"].apply(lambda x: time_to_minutes(x))

#Basic Pay
Out_DayX_df.loc[:,"Work Hours"] = Out_DayX_df.loc[:,"TO min"].astype(int) - Out_DayX_df.loc[:,"TI min"].astype(int)
Out_DayX_df.loc[:,"Work Hours"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: time_round_up(x))
Out_DayX_df.loc[:,"Basic Salary"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: work_hours(x)) * (Out_DayX_df.loc[:,"Rate Per Day"]/8)

#Overtime Pay
Out_DayX_df.loc[:,"Overtime Hours"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: overtime_hours(x))
Out_DayX_df.loc[:,"Overtime Pay"] = (Out_DayX_df.loc[:,"Overtime Hours"] * (Out_DayX_df.loc[:,"Rate Per Day"]/8) * 1.3).round(2)

#Lates
Standard_Clock_In = Standard_Clock_Hours_df.loc[0:0,"Clock In Time"].apply(lambda x: time_to_minutes(x))
Stringone = str(Standard_Clock_In)
Stringone = re.sub(r"\D",r"",Stringone)
Stringone = int(Stringone)
Out_DayX_df.loc[:,"Late Minutes"] = Out_DayX_df.loc[:,"TI min"].astype(int) - Stringone
Out_DayX_df.loc[:,"Late Hours"] = Out_DayX_df.loc[:,"Late Minutes"].apply(lambda x: late_hours(x))

#Late Deduction
Out_DayX_df.loc[:,"Late Deductions"] = (Out_DayX_df.loc[:,"Late Hours"] * (Out_DayX_df.loc[:,"Rate Per Day"]/8)).round(2)

#Gross Salary
Out_DayX_df.loc[:,"Gross Salary"] = (Out_DayX_df.loc[:,"Basic Salary"] + Out_DayX_df.loc[:,"Overtime Pay"] - Out_DayX_df.loc[:,"Late Deductions"]).round(2)

#Debt
Out_DayX_df.loc[:,"Debt"] = Out_DayX_df.loc[:,"ID Number"].apply(lambda x: debt_tracker(x))

#Miscellaneous
Out_DayX_df.loc[:,"Deductibles"] = Day_df.loc[:,"Deductibles"]
Out_DayX_df.loc[:,"Bonus"] = Day_df.loc[:,"Bonus"]

#Standing Salary
Out_DayX_df.loc[:,"Standing Salary"] = (Out_DayX_df.loc[:,"Gross Salary"] - Out_DayX_df.loc[:,"Debt"] - Out_DayX_df.loc[:,"Deductibles"] + Out_DayX_df.loc[:,"Bonus"]).round(2)

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [166]:
Out_DayX_df

,ID Number,Rate Per Day,TI min,TO min,Work Hours,Basic Salary,Overtime Hours,Overtime Pay,Late Minutes,Late Hours,Late Deductions,Gross Salary,Debt,Deductibles,Bonus,Standing Salary
0,10000000,537,460,960,8,537.0,0,0.00,-20,0,0.00,537.00,200,0,0,337.00
1,10000001,600,480,982,8,600.0,0,0.00,0,0,0.00,600.00,170,220,0,210.00
2,10000002,840,490,1040,9,840.0,1,136.50,10,1,105.00,871.50,50,80,0,741.50
3,10000003,550,480,1080,10,550.0,2,178.75,0,0,0.00,728.75,0,0,0,728.75
4,10000004,1045,482,1010,9,1045.0,1,169.81,2,1,130.62,1084.19,30,0,0,1054.19
5,10000005,729,473,960,8,729.0,0,0.00,-7,0,0.00,729.00,171,100,0,458.00
6,10000006,1255,475,1040,9,1255.0,1,203.94,-5,0,0.00,1458.94,16,130,0,1312.94
7,10000007,920,489,982,8,920.0,0,0.00,9,1,115.00,805.00,170,120,0,515.00
8,10000008,620,565,1024,8,620.0,0,0.00,85,2,155.00,465.00,50,20,0,395.00
9,10000009,845,500,1102,10,845.0,2,274.62,20,1,105.62,1014.00,110,100,0,804.00


In [102]:
# Name of the output CSV
Output_csv = Out_DayX_df.to_csv(r'C:\Users\ASUS\Desktop\Payroll System\3 Output CSVs\Out-Monday.csv')